In [1]:
import torch 
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_
from data_utils import Dictionary, Corpus

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 5
num_samples = 1000
batch_size = 20
seq_length = 30
learning_rate = 0.001

In [4]:
corpus = Corpus()
ids = corpus.get_data('train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [5]:
# RNN based Language Model
class RNNLM(nn.Module):
    def __init__(self,vocab_size,embed_size,hidden_size,num_layers):
        super(RNNLM,self).__init__()
        self.embed = nn.Embedding(vocab_size,embed_size)
        self.lstm = nn.LSTM(embed_size,hidden_size,num_layers,batch_first=True)
        self.linear = nn.Linear(hidden_size,vocab_size)

    def forward(self,x,h):
        x = self.embed(x)
        out,(h,c) = self.lstm(x,h)

        out = out.reshape(out.size(0)*out.size(1),out.size(2))

        out = self.linear(out)

        return out,(h,c)

In [6]:
model = RNNLM(vocab_size,embed_size,hidden_size,num_layers).to(device)

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
optimizer = torch.optim.Adam(model.parameters(),lr= learning_rate)

In [13]:
def detach(states):
    return [state.detach() for state in states]

In [15]:
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Forward pass
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

Epoch [1/5], Step[0/1549], Loss: 9.2151, Perplexity: 10048.06
Epoch [1/5], Step[100/1549], Loss: 6.1868, Perplexity: 486.28
Epoch [1/5], Step[200/1549], Loss: 6.0978, Perplexity: 444.89
Epoch [1/5], Step[300/1549], Loss: 5.9599, Perplexity: 387.57
Epoch [1/5], Step[400/1549], Loss: 5.8276, Perplexity: 339.54
Epoch [1/5], Step[500/1549], Loss: 5.3327, Perplexity: 207.00
Epoch [1/5], Step[600/1549], Loss: 5.3179, Perplexity: 203.95
Epoch [1/5], Step[700/1549], Loss: 5.5536, Perplexity: 258.16
Epoch [1/5], Step[800/1549], Loss: 5.3092, Perplexity: 202.18
Epoch [1/5], Step[900/1549], Loss: 5.2297, Perplexity: 186.73
Epoch [1/5], Step[1000/1549], Loss: 5.3019, Perplexity: 200.72
Epoch [1/5], Step[1100/1549], Loss: 5.4530, Perplexity: 233.45
Epoch [1/5], Step[1200/1549], Loss: 5.2585, Perplexity: 192.20
Epoch [1/5], Step[1300/1549], Loss: 5.2498, Perplexity: 190.52
Epoch [1/5], Step[1400/1549], Loss: 4.9909, Perplexity: 147.07
Epoch [1/5], Step[1500/1549], Loss: 5.2929, Perplexity: 198.91
Ep

In [16]:
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # Set intial hidden ane cell states
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))

        # Select one word id randomly
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

        for i in range(num_samples):
            # Forward propagate RNN 
            output, state = model(input, state)

            # Sample a word id
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()

            # Fill input with sampled word id for the next time step
            input.fill_(word_id)

            # File write
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, num_samples, 'sample.txt'))

Sampled [100/1000] words and save to sample.txt
Sampled [200/1000] words and save to sample.txt
Sampled [300/1000] words and save to sample.txt
Sampled [400/1000] words and save to sample.txt
Sampled [500/1000] words and save to sample.txt
Sampled [600/1000] words and save to sample.txt
Sampled [700/1000] words and save to sample.txt
Sampled [800/1000] words and save to sample.txt
Sampled [900/1000] words and save to sample.txt
Sampled [1000/1000] words and save to sample.txt


In [17]:
torch.save(model.state_dict(), 'model_language.ckpt')